In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import torch
torch.manual_seed(42)  # Set to any fixed integer


torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # If using multiple GPUs
import numpy as np
np.random.seed(42)


In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'
import numpy as np
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


import matplotlib.pyplot as plt

cuda:0


In [4]:
T_matrix=torch.tensor([[0.9, 0.1, 0.0,0.0],
        [0.00, 0.9, 0.1,0],
        [0.0, 0.00, 0.9,.1],
                      [0.0,0.0,0.0,1]], device=device) 

In [5]:
T_matrix1=torch.zeros(4,4,4)
T_matrix1[0,:,:]=T_matrix+torch.eye(4,4)*-0.0
T_matrix1[1,:,:]=T_matrix+torch.eye(4,4)*-0.3
T_matrix1[2,:,:]=T_matrix+torch.eye(4,4)*-0.5
T_matrix1[3,:,:]=T_matrix+torch.eye(4,4)*-0.7

In [6]:
T_matrix1=T_matrix1/torch.sum(T_matrix1, 2).reshape(4,4,1)

In [7]:
T_matrix1

tensor([[[0.9000, 0.1000, 0.0000, 0.0000],
         [0.0000, 0.9000, 0.1000, 0.0000],
         [0.0000, 0.0000, 0.9000, 0.1000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8571, 0.1429, 0.0000, 0.0000],
         [0.0000, 0.8571, 0.1429, 0.0000],
         [0.0000, 0.0000, 0.8571, 0.1429],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8000, 0.2000, 0.0000, 0.0000],
         [0.0000, 0.8000, 0.2000, 0.0000],
         [0.0000, 0.0000, 0.8000, 0.2000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.6667, 0.3333, 0.0000, 0.0000],
         [0.0000, 0.6667, 0.3333, 0.0000],
         [0.0000, 0.0000, 0.6667, 0.3333],
         [0.0000, 0.0000, 0.0000, 1.0000]]])

In [8]:

s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)



print(T_matrix)

print(s_0)

tensor([[0.9000, 0.1000, 0.0000, 0.0000],
        [0.0000, 0.9000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.1000],
        [0.0000, 0.0000, 0.0000, 1.0000]])
tensor([1., 0., 0.])


In [9]:
import numpy as np

# Define the joint state space for two units (4 states each)
joint_states = [(i, j) for i in range(4) for j in range(4)]
num_joint_states = len(joint_states)
# Sort the joint states based on the sum of values in each tuple
joint_states = sorted(joint_states, key=lambda x: sum(x))
# Example: Define a 4x4 joint transition matrix P((s1, s2) -> (s1', s2'))
P_joint = np.array([
    # Each row corresponds to a joint state, each column to the next joint state
    [0.4, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],  # (0,0) -> various states
    [0.3, 0.3, 0.1, 0.1, 0.07, 0.05, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (0,1)
    [0.2, 0.2, 0.3, 0.1, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.03, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,2)
    [0.1, 0.1, 0.2, 0.4, 0.05, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,3)
    [0.3, 0.2, 0.1, 0.1, 0.2, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (1,0)
    [0.2, 0.2, 0.1, 0.1, 0.3, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0],  # (1,1)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.4, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.0, 0.0],  # (1,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.2, 0.1, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0],  # (1,3)
    [0.3, 0.1, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01],  # (2,0)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.1, 0.4, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01],  # (2,1)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.05, 0.05, 0.3, 0.05, 0.02, 0.02, 0.01, 0.01],  # (2,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.1, 0.3, 0.05, 0.05, 0.02, 0.02],  # (2,3)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.3, 0.1, 0.05, 0.05],  # (3,0)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.3, 0.05, 0.1],  # (3,1)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.05, 0.1, 0.4, 0.2],  # (3,2)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.1, 0.2, 0.3],  # (3,3)
])

# Set lower triangular elements to 0
for i in range(16):
    for j in range(i):
        P_joint[i, j] = 0
# Set lower triangular elements to 0
for i in range(16):
    for j in range(i+5,16):
        P_joint[i, j] = 0

P_joint=P_joint+np.eye(16,16)*0.5
# Normalize each row so the sum of probabilities equals 1
P_joint = P_joint / P_joint.sum(axis=1, keepdims=True)

# Check if the rows sum to 1
row_sums = P_joint.sum(axis=1)
print(row_sums)  # Should all be close to 1


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [25]:

# import torch
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(42)
#     torch.cuda.manual_seed_all(42)  # For multi-GPU environments


In [29]:
import numpy as np
import torch
from hmmlearn import hmm

# Load data
pred1 = torch.load('DATA_NON_LINEAR_8_MACHINES.pth')

# Extract observations and underlying states for 8 machines
# Each pred1[i] = [observations, 501, states]
observations_all = [item[0].numpy() for item in pred1]  # shape (n_samples, 8)
true_states_all = [item[2].numpy() for item in pred1]   # shape (n_samples, 8)

n_machines = 8

# Prepare data for each machine
observations_per_machine = [
    [obs[:, m].reshape(-1, 1) for obs in observations_all]
    for m in range(n_machines)
]
true_states_per_machine = [
    [states[:, m].reshape(-1, 1) for states in true_states_all]
    for m in range(n_machines)
]

def train_hmm_and_evaluate(observations, n_components=4, covariance_type="spherical", n_iter=10000):
    lengths = [len(seq) for seq in observations]
    X = np.concatenate(observations)

    model = hmm.GaussianHMM(n_components=n_components, covariance_type=covariance_type, n_iter=n_iter)
    model.fit(X, lengths)

    log_likelihoods = [model.score(seq) for seq in observations]
    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood = np.std(log_likelihoods)

    filtered_states = [model.predict(seq) for seq in observations]

    return model, mean_likelihood, std_likelihood, filtered_states

def calculate_filtration_error(filtered_states, true_states):
    total_correct = 0
    total_states = 0
    for filtered, true in zip(filtered_states, true_states):
        total_correct += (filtered.reshape(-1, 1) == true).sum()
        total_states += len(true)
    return total_correct / total_states

# Train HMMs for all 8 machines and collect results
results = []
for m in range(n_machines):
    model, mean_lik, std_lik, filtered_states = train_hmm_and_evaluate(observations_per_machine[m])
    error = calculate_filtration_error(filtered_states, true_states_per_machine[m])
    results.append((m+1, mean_lik, std_lik, error))

# Print results
for m, mean_lik, std_lik, error in results:
    print(f"Machine {m}: Mean Likelihood = {mean_lik:.4f}, Std = {std_lik:.4f}, Filtration Accuracy = {error:.4f}")


Machine 1: Mean Likelihood = -869.3511, Std = 227.6844, Filtration Accuracy = 0.4309
Machine 2: Mean Likelihood = -1230.4739, Std = 28.4815, Filtration Accuracy = 0.4231
Machine 3: Mean Likelihood = -709.2651, Std = 165.3523, Filtration Accuracy = 0.3619
Machine 4: Mean Likelihood = -666.6343, Std = 143.4310, Filtration Accuracy = 0.3920
Machine 5: Mean Likelihood = -652.7685, Std = 140.2242, Filtration Accuracy = 0.3952
Machine 6: Mean Likelihood = -1234.9161, Std = 29.0855, Filtration Accuracy = 0.7911
Machine 7: Mean Likelihood = -1230.3494, Std = 29.3901, Filtration Accuracy = 0.0031
Machine 8: Mean Likelihood = -1240.3780, Std = 23.0598, Filtration Accuracy = 0.0007


In [30]:
def rearrange_states(filtered_states, hmm_means):
    """
    Rearranges states based on the order of HMM means.
    
    Args:
        filtered_states (list of np.ndarray): List of filtered states.
        hmm_means (np.ndarray): The means of the Gaussian HMM components.
        
    Returns:
        list of np.ndarray: Rearranged filtered states.
    """
    # Get the new state order by sorting the means
    sorted_indices = np.argsort(hmm_means.flatten())
    state_mapping = {old: new for new, old in enumerate(sorted_indices)}

    # Rearrange states in the filtered results
    rearranged_states = []
    for states in filtered_states:
        rearranged_states.append(np.vectorize(state_mapping.get)(states))
    
    return rearranged_states

In [35]:
import time
import numpy as np
from hmmlearn import hmm
import torch



def train_hmm_and_evaluate(observations, n_components=4, covariance_type="spherical", n_iter=10000):
    lengths = [len(seq) for seq in observations]
    X = np.concatenate(observations)

    model = hmm.GaussianHMM(n_components=n_components, covariance_type=covariance_type, n_iter=n_iter)
    model.fit(X, lengths)

    log_likelihoods = [model.score(seq) for seq in observations]
    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood = np.std(log_likelihoods)

    filtered_states = [model.predict(seq) for seq in observations]

    return model, mean_likelihood, std_likelihood, filtered_states

def calculate_filtration_error(filtered_states, true_states):
    total_correct = 0
    total_states = 0
    for filtered, true in zip(filtered_states, true_states):
        total_correct += (filtered.reshape(-1, 1) == true).sum()
        total_states += len(true)
    return total_correct / total_states

# Load data
pred1 = torch.load('DATA_NON_LINEAR_8_MACHINES.pth')

# Each pred1[i] = [observations, 501, states]
observations_all = [item[0].numpy() for item in pred1]
true_states_all = [item[2].numpy() for item in pred1]

n_machines = 8
n_repeats = 10
results_all = []
time_all = []

# Prepare data for each machine
observations_per_machine = [
    [obs[:, m].reshape(-1, 1) for obs in observations_all]
    for m in range(n_machines)
]
true_states_per_machine = [
    [states[:, m].reshape(-1, 1) for states in true_states_all]
    for m in range(n_machines)
]

for m in range(n_machines):
    results_m = []
    time_m = []
    for _ in range(n_repeats):
        start_time = time.time()
        model, mean_likelihood, std_likelihood, filtered_states = train_hmm_and_evaluate(observations_per_machine[m])
        hmm_means = model.means_
        filtered_states = rearrange_states(filtered_states, hmm_means)
        elapsed_time = time.time() - start_time
        error = calculate_filtration_error(filtered_states, true_states_per_machine[m])
        results_m.append((mean_likelihood, std_likelihood, error))
        time_m.append(elapsed_time)
    results_all.append(results_m)
    time_all.append(time_m)

# Calculate mean and std for metrics and time for each machine
mean_metrics_all = [np.mean(np.array(results), axis=0) for results in results_all]
std_metrics_all = [np.std(np.array(results), axis=0) for results in results_all]
mean_time_all = [np.mean(times) for times in time_all]
std_time_all = [np.std(times) for times in time_all]

# Prepare output string
output_lines = []
for m in range(n_machines):
    output_lines.append(f"Machine {m+1} HMM Results:")
    output_lines.append(f"  Mean Likelihood: {mean_metrics_all[m][0]:.4f}, Std: {std_metrics_all[m][0]:.4f}")
    output_lines.append(f"  Mean Filtration Accuracy: {mean_metrics_all[m][2]:.4f}, Std: {std_metrics_all[m][2]:.4f}")
    output_lines.append(f"  Mean Computation Time: {mean_time_all[m]:.4f}s, Std: {std_time_all[m]:.4f}s\n")

# Save to result.txt
with open('result_HMM.txt', 'w') as f:
    f.write('\n'.join(output_lines))

# Print final summary to console as well
for line in output_lines:
    print(line)


Machine 1 HMM Results:
  Mean Likelihood: -870.1806, Std: 76.1045
  Mean Filtration Accuracy: 0.7350, Std: 0.2395
  Mean Computation Time: 10.7883s, Std: 10.8219s

Machine 2 HMM Results:
  Mean Likelihood: -1215.2039, Std: 20.3009
  Mean Filtration Accuracy: 0.7223, Std: 0.3504
  Mean Computation Time: 15.7606s, Std: 11.8490s

Machine 3 HMM Results:
  Mean Likelihood: -704.1236, Std: 10.6698
  Mean Filtration Accuracy: 0.7672, Std: 0.1828
  Mean Computation Time: 10.9705s, Std: 8.3075s

Machine 4 HMM Results:
  Mean Likelihood: -692.9789, Std: 13.3897
  Mean Filtration Accuracy: 0.7058, Std: 0.2390
  Mean Computation Time: 7.2397s, Std: 5.5580s

Machine 5 HMM Results:
  Mean Likelihood: -734.8101, Std: 113.7737
  Mean Filtration Accuracy: 0.8613, Std: 0.1809
  Mean Computation Time: 8.7942s, Std: 9.6360s

Machine 6 HMM Results:
  Mean Likelihood: -1214.0866, Std: 19.9149
  Mean Filtration Accuracy: 0.6509, Std: 0.3467
  Mean Computation Time: 12.8189s, Std: 11.7075s

Machine 7 HMM Resu